In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  

In [2]:
pair = "USDCHF"
NB_H1_CANDLES = 10000

In [16]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [17]:
rates = Rates(pair, 13*2000, mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [18]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
0,2021-08-12 11:00:00,0.92135,0.92208,0.92118,0.92135,0.92135,0.92135,18.888889,18.888889,0.000000,...,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-08-12 12:00:00,0.92136,0.92181,0.92121,0.92174,0.92136,0.92174,25.000000,88.333333,63.333333,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-08-12 13:00:00,0.92173,0.92250,0.92171,0.92204,0.92173,0.92204,2.531646,41.772152,39.240506,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-08-12 14:00:00,0.92204,0.92251,0.92186,0.92199,0.92199,0.92204,20.000000,27.692308,7.692308,...,63.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-08-12 15:00:00,0.92201,0.92306,0.92159,0.92218,0.92201,0.92218,28.571429,40.136054,11.564626,...,39.240506,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [19]:
df_an = RSI(df_an)
df_an = BollingerBands(df_an)
df_an = ATR(df_an)
df_an = KeltnerChannels(df_an)
df_an = MACD(df_an)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,BB_MA,BB_UP,BB_LW,ATR_14,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST
0,2021-08-12 11:00:00,0.92135,0.92208,0.92118,0.92135,0.92135,0.92135,18.888889,18.888889,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-12 12:00:00,0.92136,0.92181,0.92121,0.92174,0.92136,0.92174,25.000000,88.333333,63.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-12 13:00:00,0.92173,0.92250,0.92171,0.92204,0.92173,0.92204,2.531646,41.772152,39.240506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-12 14:00:00,0.92204,0.92251,0.92186,0.92199,0.92199,0.92204,20.000000,27.692308,7.692308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-12 15:00:00,0.92201,0.92306,0.92159,0.92218,0.92201,0.92218,28.571429,40.136054,11.564626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2023-03-23 06:00:00,0.91569,0.91601,0.91512,0.91522,0.91522,0.91569,11.235955,64.044944,52.808989,...,0.919547,0.925915,0.913180,0.001970,0.918652,0.921340,0.915964,-0.002063,-0.001728,-0.000335
9995,2023-03-23 07:00:00,0.91522,0.91562,0.91492,0.91562,0.91522,0.91562,42.857143,100.000000,57.142857,...,0.919234,0.925779,0.912689,0.001871,0.918363,0.920559,0.916167,-0.002048,-0.001792,-0.000256
9996,2023-03-23 08:00:00,0.91559,0.91600,0.91407,0.91522,0.91522,0.91559,59.585492,78.756477,19.170984,...,0.918878,0.925514,0.912242,0.001909,0.918064,0.920494,0.915634,-0.002046,-0.001843,-0.000203
9997,2023-03-23 09:00:00,0.91521,0.91663,0.91376,0.91606,0.91521,0.91606,50.522648,80.139373,29.616725,...,0.918539,0.925141,0.911938,0.002004,0.917873,0.920677,0.915069,-0.001953,-0.001865,-0.000088


In [20]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.00015000000000000001

In [38]:
df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an.pivots_l = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an.pivots_h = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,low_perv,high_perv,low_next,high_next,BUY_SIG,SELL_SIG,SIGNAL,SIGNAL_LABEL,TP,SL
0,2021-08-13 21:00:00,0.91533,0.91557,0.91507,0.91510,0.91510,0.91533,6.000000,52.000000,46.000000,...,0.91506,0.91565,0.91499,0.91544,0,0,0,0,0.0,0.0
1,2021-08-13 22:00:00,0.91509,0.91544,0.91499,0.91507,0.91507,0.91509,17.777778,22.222222,4.444444,...,0.91507,0.91557,0.91503,0.91524,0,0,0,0,0.0,0.0
2,2021-08-13 23:00:00,0.91508,0.91524,0.91503,0.91512,0.91508,0.91512,23.809524,42.857143,19.047619,...,0.91499,0.91544,0.91489,0.91544,0,0,0,0,0.0,0.0
3,2021-08-16 00:00:00,0.91508,0.91544,0.91489,0.91544,0.91508,0.91544,34.545455,100.000000,65.454545,...,0.91503,0.91524,0.91486,0.91566,0,0,0,0,0.0,0.0
4,2021-08-16 01:00:00,0.91529,0.91566,0.91486,0.91537,0.91529,0.91537,53.750000,63.750000,10.000000,...,0.91489,0.91544,0.91522,0.91544,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9958,2023-03-23 04:00:00,0.91693,0.91730,0.91561,0.91585,0.91585,0.91693,14.201183,78.106509,63.905325,...,0.91580,0.91720,0.91448,0.91611,0,0,0,0,0.0,0.0
9959,2023-03-23 05:00:00,0.91585,0.91611,0.91448,0.91569,0.91569,0.91585,74.233129,84.049080,9.815951,...,0.91561,0.91730,0.91512,0.91601,0,0,0,0,0.0,0.0
9960,2023-03-23 06:00:00,0.91569,0.91601,0.91512,0.91522,0.91522,0.91569,11.235955,64.044944,52.808989,...,0.91448,0.91611,0.91492,0.91562,0,0,0,0,0.0,0.0
9961,2023-03-23 07:00:00,0.91522,0.91562,0.91492,0.91562,0.91522,0.91562,42.857143,100.000000,57.142857,...,0.91512,0.91601,0.91407,0.91600,0,0,0,0,0.0,0.0


In [39]:
BUY = 1
SELL = -1
NONE = 0

df_an['BUY_SIG'] = 0
df_an['SELL_SIG'] = 0
df_an['SIGNAL'] = 0
df_an['SIGNAL_LABEL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0

In [40]:
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.BUY_SIG[i] = 1
                    df_an.SIGNAL[i] = BUY
                    df_an.SIGNAL_LABEL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] #- SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i] #+ SPREAD
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SELL_SIG[i] = 1
                    df_an.SIGNAL[i] = SELL
                    df_an.SIGNAL_LABEL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] #+ SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_2128\2220319849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SELL_SIG[i] = 1
C:\Users\meysam\AppData\Local\Temp\ipykernel_2128\2220319849.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = SELL
C:\Users\meysam\AppData\Local\Temp\ipykernel_2128\2220319849.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL_LABEL[i] = SELL
C:\Users\meysam\AppData

In [41]:
df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

In [42]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST', 'low_perv', 'high_perv', 'low_next', 'high_next',
       'BUY_SIG', 'SELL_SIG', 'SIGNAL', 'SIGNAL_LABEL', 'TP', 'SL'],
      dtype='object')

In [43]:
df_features = df_an#[feature_col]
# mu = df_features.mean()
# std = df_features.std()
# df_features = (df_features-mu)/std

In [44]:
df_features.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9963 entries, 0 to 9962
Data columns (total 52 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   time              9963 non-null   datetime64[ns]
 1   open              9963 non-null   float64       
 2   high              9963 non-null   float64       
 3   low               9963 non-null   float64       
 4   close             9963 non-null   float64       
 5   body_lower        9963 non-null   float64       
 6   body_upper        9963 non-null   float64       
 7   body_bottom_perc  9963 non-null   float64       
 8   body_top_perc     9963 non-null   float64       
 9   body_perc         9963 non-null   float64       
 10  direction         9963 non-null   int64         
 11  body_size         9963 non-null   float64       
 12  low_change        9963 non-null   float64       
 13  high_change       9963 non-null   float64       
 14  body_size_change  9963 n

In [50]:
X = []
Y = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
#     'pivots_h','pivots_h'
#     'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
for i in range(4,len(df_an)):
    feature = []
    feature.append(df_an.pivots_h[i-1])
    feature.append(df_an.pivots_l[i-1])
    for fc in feature_col:
        feature.append(df_an[fc][i])
        
    for fc in feature_col:
        feature.append(df_an[fc][i-1])
        feature.append(df_an[fc][i-2])
        feature.append(df_an[fc][i-3])
        feature.append(df_an[fc][i-4])
        
    X.append(feature)
    Y.append(df_an.SIGNAL[i])

In [51]:
df_an.pivots_h

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9960    0.0
9961    0.0
9962    0.0
9963    0.0
9964    0.0
Length: 9965, dtype: float64

In [52]:
len(feature)

192

In [57]:
X_train = X[:8000]
Y_train = Y[:8000]
X_test = X[8000:]
Y_test = Y[8000:]


In [114]:
# X_train = np.array(X_train)

In [115]:
# X_train = (X_train - X_train.mean(axis=0))/X_train.std(axis=0)

In [80]:
# X_train_extend = []
# Y_train_extend = []
# for i in range(8000):
#     if Y_train[i] != NONE:
#         for j in range(22):
#             X_train_extend.append(X[i])
#             Y_train_extend.append(Y[j])
#     X_train_extend.append(X[i])
#     Y_train_extend.append(Y[j])

In [81]:
p = np.random.permutation(len(X_train_extend))

In [82]:
# X_train_extend = np.array(X_train_extend)
# Y_train_extend = np.array(Y_train_extend)

In [83]:
# X_train_extend = X_train_extend[p]
# Y_train_extend = Y_train_extend[p]

In [84]:
# scaler = StandardScaler()  
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test) 


In [53]:
df_an.SIGNAL.value_counts()

 0    9017
 1     482
-1     464
Name: SIGNAL, dtype: int64

In [54]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network._multilayer_perceptron:

class MLPClassifier(sklearn.base.ClassifierMixin, BaseMultilayerPerceptron)
 |  MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity

In [58]:
clf = MLPClassifier( solver='sgd', alpha=1e-5,learning_rate_init=1e-5,
                     tol=0.0000010, hidden_layer_sizes=(200,200), 
                    random_state=1,max_iter=100000,verbose=True,warm_start = True)
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [59]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 1.46331720
Iteration 2, loss = 0.97985391
Iteration 3, loss = 0.82466307
Iteration 4, loss = 0.75213366
Iteration 5, loss = 0.70687142
Iteration 6, loss = 0.67184430
Iteration 7, loss = 0.64433352
Iteration 8, loss = 0.62200161
Iteration 9, loss = 0.60311928
Iteration 10, loss = 0.58736125
Iteration 11, loss = 0.57212191
Iteration 12, loss = 0.55958271
Iteration 13, loss = 0.54708737
Iteration 14, loss = 0.53459855
Iteration 15, loss = 0.52515745
Iteration 16, loss = 0.51717373
Iteration 17, loss = 0.50853868
Iteration 18, loss = 0.50040902
Iteration 19, loss = 0.49302217
Iteration 20, loss = 0.48813395
Iteration 21, loss = 0.48229938
Iteration 22, loss = 0.47532804
Iteration 23, loss = 0.47051624
Iteration 24, loss = 0.46636703
Iteration 25, loss = 0.46061895
Iteration 26, loss = 0.45628523
Iteration 27, loss = 0.45138885
Iteration 28, loss = 0.44728942
Iteration 29, loss = 0.44445096
Iteration 30, loss = 0.44030610
Iteration 31, loss = 0.43711351
Iteration 32, los

Iteration 253, loss = 0.30063616
Iteration 254, loss = 0.30014251
Iteration 255, loss = 0.30112222
Iteration 256, loss = 0.30109246
Iteration 257, loss = 0.29996022
Iteration 258, loss = 0.29986676
Iteration 259, loss = 0.30006134
Iteration 260, loss = 0.29984109
Iteration 261, loss = 0.30026453
Iteration 262, loss = 0.29959651
Iteration 263, loss = 0.29865243
Iteration 264, loss = 0.29990463
Iteration 265, loss = 0.29874665
Iteration 266, loss = 0.29837181
Iteration 267, loss = 0.29813485
Iteration 268, loss = 0.29796314
Iteration 269, loss = 0.29791343
Iteration 270, loss = 0.29771458
Iteration 271, loss = 0.29721914
Iteration 272, loss = 0.29718424
Iteration 273, loss = 0.29727468
Iteration 274, loss = 0.29661138
Iteration 275, loss = 0.29682853
Iteration 276, loss = 0.29631938
Iteration 277, loss = 0.29634159
Iteration 278, loss = 0.29617683
Iteration 279, loss = 0.29649848
Iteration 280, loss = 0.29661590
Iteration 281, loss = 0.29564471
Iteration 282, loss = 0.29600293
Iteration 

Iteration 502, loss = 0.27054598
Iteration 503, loss = 0.27076270
Iteration 504, loss = 0.27060316
Iteration 505, loss = 0.27091202
Iteration 506, loss = 0.27023593
Iteration 507, loss = 0.27049339
Iteration 508, loss = 0.27045821
Iteration 509, loss = 0.26999021
Iteration 510, loss = 0.27092619
Iteration 511, loss = 0.27106767
Iteration 512, loss = 0.27037375
Iteration 513, loss = 0.27029711
Iteration 514, loss = 0.27003259
Iteration 515, loss = 0.27045881
Iteration 516, loss = 0.26985409
Iteration 517, loss = 0.26950678
Iteration 518, loss = 0.26956017
Iteration 519, loss = 0.26986941
Iteration 520, loss = 0.26974307
Iteration 521, loss = 0.26974924
Iteration 522, loss = 0.26979163
Iteration 523, loss = 0.26941828
Iteration 524, loss = 0.26902709
Iteration 525, loss = 0.26892805
Iteration 526, loss = 0.26866261
Iteration 527, loss = 0.26886824
Iteration 528, loss = 0.26902489
Iteration 529, loss = 0.26830687
Iteration 530, loss = 0.26892333
Iteration 531, loss = 0.26852191
Iteration 

Iteration 751, loss = 0.25469727
Iteration 752, loss = 0.25465074
Iteration 753, loss = 0.25506434
Iteration 754, loss = 0.25452979
Iteration 755, loss = 0.25463601
Iteration 756, loss = 0.25436291
Iteration 757, loss = 0.25515717
Iteration 758, loss = 0.25405277
Iteration 759, loss = 0.25437356
Iteration 760, loss = 0.25442728
Iteration 761, loss = 0.25475603
Iteration 762, loss = 0.25389603
Iteration 763, loss = 0.25388796
Iteration 764, loss = 0.25441934
Iteration 765, loss = 0.25391034
Iteration 766, loss = 0.25390706
Iteration 767, loss = 0.25420442
Iteration 768, loss = 0.25440581
Iteration 769, loss = 0.25448949
Iteration 770, loss = 0.25395345
Iteration 771, loss = 0.25431820
Iteration 772, loss = 0.25417544
Iteration 773, loss = 0.25381338
Iteration 774, loss = 0.25354652
Iteration 775, loss = 0.25333901
Iteration 776, loss = 0.25407254
Iteration 777, loss = 0.25402382
Iteration 778, loss = 0.25347823
Iteration 779, loss = 0.25337220
Iteration 780, loss = 0.25301588
Iteration 

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(200, 200),
              learning_rate_init=1e-05, max_iter=100000, random_state=1,
              solver='sgd', tol=1e-06, verbose=True, warm_start=True)

In [32]:
y_perdict = clf.predict(X_test)

In [33]:
confusion_matrix(Y_test, y_perdict)

array([[   0,   87,    0],
       [   0, 1800,    0],
       [   0,   78,    0]], dtype=int64)

In [ ]:
Y_test

In [ ]:
y_perdict = list(y_perdict)

In [ ]:
len(y_perdict)

In [ ]:
df_test = df_an.iloc[8000:].copy()
df_test.reset_index(drop=True, inplace=True)
# df_test["SIGNAL"] = y_perdict

In [ ]:
df_test.SIGNAL

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

In [ ]:
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):# start from zero!!!!!!!!!!!!!!!!!!!!!
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['TP'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5.result.value_counts()

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])